<a href="https://colab.research.google.com/github/WilderJoseth/statistics_public/blob/master/projects/python/TitanicGLMAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introduction

This work will focus on applying statistical skills to understand Titanic data using logistic regression methods with Generalized Linear Model.

Version: 1.0

# 2. Data Understanding

## 2.1. Libraries

In [105]:
# Tools for data exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 2.2. Load data

In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes


In [ ]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 22.4MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 29.3MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.21MB/s]


In [106]:
dsTrain = pd.read_csv('/content/train.csv')
# dsTest = pd.read_csv('/content/test.csv')

## 2.3. Process data

In [107]:
print('Shape:', dsTrain.shape)
dsTrain.head()

Shape: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [108]:
dsTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [109]:
dsTrain.describe(include = 'all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Bonnell, Miss. Elizabeth",male,NaN,NaN,NaN,1601,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


First conclusions:
* Remove PassengerId feature, because this is an incremental variable. All observations are unique.
* Remove Name feature, because this add only noise. Sex and Age are enough to explain a person's characteristic.
* Join SibSp and Parch features, and transform them in a binary variable.
* Fill Age feature with the mean.
* Remove Ticket feature, because this add only noise. Most of its observations are unique.
* Remove Cabin feature, because most of its observations are missing.

### 2.3.1. Fix missing values

In [110]:
dsTrain['Age'] = dsTrain['Age'].fillna(dsTrain['Age'].median())

In [111]:
dsTrain['Embarked'] = dsTrain['Embarked'].fillna('S')

### 2.3.2. Create new variable

In [112]:
dsTrain['FamilySize'] =  dsTrain['SibSp'] + dsTrain['Parch']
dsTrain['IsAlone'] = dsTrain['FamilySize'].map(lambda x: 0 if x > 0 else 1)

### 2.3.3. Remove variables

In [113]:
dsTrain = dsTrain.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'FamilySize'], axis = 1)

In [114]:
print('Shape:', dsTrain.shape)
dsTrain.head()

Shape: (891, 7)


,Survived,Pclass,Sex,Age,Fare,Embarked,IsAlone
0,0,3,male,22.0,7.2500,S,0
1,1,1,female,38.0,71.2833,C,0
2,1,3,female,26.0,7.9250,S,1
3,1,1,female,35.0,53.1000,S,0
4,0,3,male,35.0,8.0500,S,1


### 2.3.4. Transform variables

In [115]:
from sklearn import preprocessing

In [116]:
# Encode Pclass variable
# 1 -> 0
# 2 -> 1
# 3 -> 2
le = preprocessing.LabelEncoder()
le.fit([1, 2, 3])
dsTrain['Pclass_EC'] = le.transform(dsTrain['Pclass'].values)

In [117]:
# Encode Sex variable
# female -> 0
# male -> 1
le = preprocessing.LabelEncoder()
le.fit(['female', 'male'])
dsTrain['Sex_EC'] = le.transform(dsTrain['Sex'].values)

In [118]:
# Encode Embarked variable
# C -> 0
# Q -> 1
# S -> 2
le = preprocessing.LabelEncoder()
le.fit(['S', 'C', 'Q'])
dsTrain['Embarked_EC'] = le.transform(dsTrain['Embarked'].values)

In [119]:
dsTrain.head(10)

,Survived,Pclass,Sex,Age,Fare,Embarked,IsAlone,Pclass_EC,Sex_EC,Embarked_EC
0,0,3,male,22.0,7.2500,S,0,2,1,2
1,1,1,female,38.0,71.2833,C,0,0,0,0
2,1,3,female,26.0,7.9250,S,1,2,0,2
3,1,1,female,35.0,53.1000,S,0,0,0,2
4,0,3,male,35.0,8.0500,S,1,2,1,2
5,0,3,male,28.0,8.4583,Q,1,2,1,1
6,0,1,male,54.0,51.8625,S,1,0,1,2
7,0,3,male,2.0,21.0750,S,0,2,1,2
8,1,3,female,27.0,11.1333,S,0,2,0,2
9,1,2,female,14.0,30.0708,C,0,1,0,0


In [120]:
dsTrain = dsTrain.drop(['Pclass', 'Sex', 'Embarked'], axis = 1)

In [121]:
dsTrain['IsAlone'] = dsTrain['IsAlone'].astype('category')
dsTrain['Pclass_EC'] = dsTrain['Pclass_EC'].astype('category')
dsTrain['Sex_EC'] = dsTrain['Sex_EC'].astype('category')
dsTrain['Embarked_EC'] = dsTrain['Embarked_EC'].astype('category')

In [122]:
dsTrain.head()

,Survived,Age,Fare,IsAlone,Pclass_EC,Sex_EC,Embarked_EC
0,0,22.0,7.2500,0,2,1,2
1,1,38.0,71.2833,0,0,0,0
2,1,26.0,7.9250,1,2,0,2
3,1,35.0,53.1000,0,0,0,2
4,0,35.0,8.0500,1,2,1,2


In [123]:
dsTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Survived     891 non-null    int64   
 1   Age          891 non-null    float64 
 2   Fare         891 non-null    float64 
 3   IsAlone      891 non-null    category
 4   Pclass_EC    891 non-null    category
 5   Sex_EC       891 non-null    category
 6   Embarked_EC  891 non-null    category
dtypes: category(4), float64(2), int64(1)
memory usage: 24.9 KB


## 2.4. Logistic regression

### 2.4.1. Generalized Linear Model
Hypeparameters:
* **Significance level**: 0.05

In [124]:
import statsmodels.api as sm

In [131]:
model = sm.GLM.from_formula('Survived ~ Age + Fare + IsAlone + Pclass_EC + Sex_EC + Embarked_EC', family = sm.families.Binomial(), data = dsTrain)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                            GLM   Df Residuals:                      882
Model Family:                Binomial   Df Model:                            8
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -399.10
Date:                Wed, 19 Aug 2020   Deviance:                       798.19
Time:                        02:12:01   Pearson chi2:                     933.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            3.7891      0.456      8.308      0.000       2.895       4.683
IsAlone[T.1]         0.0719      0.196      0.366      0.714      -0.313       0.456
Pclass_EC[T.1]      -0.9464      0.290     -3.258      0.001      -1.516      -0.377
Pclass_EC[T.2]      -2.2779      0.289     -7.888      0.000      -2.844      -1.712
Sex_EC[T.1]         -2.5953      0.196    -13.227      0.000      -2.980      -2.211
Embarked_EC[T.1]    -0.0238      0.373     -0.064      0.949      -0.755       0.708
Embarked_EC[T.2]    -0.5403      0.236     -2.291      0.022      -1.003      -0.078
Age                 -0.0332      0.008     -4.389      0.000      -0.048      -0.018
Fare                 0.0004      0.002      0.175      0.861      -0.004       0.005
====================================================================================
"""

**Observations**

Parameters:
* IsAlone variable is not statistical significance.
* Embarked_EC variable is not statistical significance.
* Fare is not variable statistical significance.

These variables are not statistical significance because their p-value is greater than significance level, so I will remove them from the model.

In [138]:
# Calculate model after removing not statistical significance variables
model = sm.GLM.from_formula('Survived ~ Age + Pclass_EC + Sex_EC', family = sm.families.Binomial(), data = dsTrain)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                            GLM   Df Residuals:                      886
Model Family:                Binomial   Df Model:                            4
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -402.76
Date:                Wed, 19 Aug 2020   Deviance:                       805.53
Time:                        02:34:28   Pearson chi2:                     930.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          3.5308      0.364      9.702      0.000       2.817       4.244
Pclass_EC[T.1]    -1.1154      0.257     -4.335      0.000      -1.620      -0.611
Pclass_EC[T.2]    -2.3342      0.242     -9.665      0.000      -2.808      -1.861
Sex_EC[T.1]       -2.6115      0.187    -13.993      0.000      -2.977      -2.246
Age               -0.0332      0.007     -4.491      0.000      -0.048      -0.019
==================================================================================
"""

**Observations**

Formula: `log(y) = 3.5308 - 1.1154*Pclass_EC_1 - 2.3342*Pclass_EC_2 - 2.6115*Sex_EC_1 - 0.0332*Age`

**Parameters Intepretations**

**Pclass**
* The model suggests that class 1 (original value) is most likely to survive than other classes keeping all else constant, because this category is a reference level and other categories have negative log odds.

**Sex**
* The model suggests that log odds of survival for males decrease by 2.6115 keeping all else constant.
* Calculating odds ratio `exp(-2.6115) = 0.0734`, I can say for for males, odds of survival is 0.0734 times that females. As odds ratio (male / female) is less than zero, I can say that males are less likely to survive than females.

**Age**
* The model suggests that log odds of survival for older people decrease 0.0332 by year. keeping all else constant.
* Calculating odds ratio `exp(-0.0332) = 0.96734`, I can say for older people, odds survival is 0.96734 times that younger people. Another point, odds ratio is close to `1`, so that might mean that this variable is not a good predictor, because to get `1` evidence for young people and old people should be similar.

# 3. References

https://stats.idre.ucla.edu/stata/faq/how-do-i-interpret-odds-ratios-in-logistic-regression/